# Finding Similar Music Using `Implicit` and Alternating Least Squares (ALS) for Matrix Factorization

This follow's Ethan Rosenthal's [Intro to Implicit Matrix Factorization: Classic ALS with Sketchfab Models](https://www.ethanrosenthal.com/2016/10/19/implicit-mf-part-1]).

## Preliminaries

**Next two lines are for pretty output for all prints in a Pandas cell, not just the last.**

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Imports

In [4]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.sparse as sparse
import pickle
import csv
# import implicit
from implicit.als import AlternatingLeastSquares
from implicit.approximate_als import (AnnoyAlternatingLeastSquares,
                                      FaissAlternatingLeastSquares,
                                      NMSLibAlternatingLeastSquares)
from implicit.bpr import BayesianPersonalizedRanking
from implicit.datasets.lastfm import get_lastfm
from implicit.datasets.movielens import get_movielens
from implicit.datasets.reddit import get_reddit
from implicit.datasets.sketchfab import get_sketchfab
from implicit.datasets.million_song_dataset import get_msd_taste_profile
from implicit.lmf import LogisticMatrixFactorization
from implicit.nearest_neighbours import (BM25Recommender, CosineRecommender,
                                         TFIDFRecommender, bm25_weight)
from sklearn.metrics import mean_squared_error

import itertools
import copy
%matplotlib inline

plt.style.use('ggplot')

### Initializations

In [5]:
# maps command line model argument to class name
models = {"als":  AlternatingLeastSquares,
          "nmslib_als": NMSLibAlternatingLeastSquares,
          "annoy_als": AnnoyAlternatingLeastSquares,
          "faiss_als": FaissAlternatingLeastSquares,
          "tfidf": TFIDFRecommender,
          "cosine": CosineRecommender,
          "bpr": BayesianPersonalizedRanking,
          "lmf": LogisticMatrixFactorization,
          "bm25": BM25Recommender}

dataSets = {"lastfm": get_lastfm,
            "movielens": get_movielens,
            "reddit": get_reddit,
            "sketchfab": get_sketchfab,
            "million_song": get_msd_taste_profile}

## Functions

### `getModel()`

In [6]:
def getModel(modelName):
    """
    This merely fetches a particular implicit model type
    """
    print("getting model %s" % modelName)
    modelClass = models.get(modelName)
    if not modelClass:
        raise ValueError("Unknown Model '%s'" % modelName)

    # some default params
    if issubclass(modelClass, AlternatingLeastSquares):
        params = {'factors': 16, 'dtype': np.float32}
    elif modelName == "bm25":
        params = {'K1': 100, 'B': 0.5}
    elif modelName == "bpr":
        params = {'factors': 63}
    elif modelName == "lmf":
        params = {'factors': 30, "iterations": 40, "regularization": 1.5}
    else:
        params = {}

    return modelClass(**params)

### `getDatums()`

In [7]:
def getDatums(dataset):
    """
    No real work to do here, since the heavy lifting is done by the implicit.dataSets module
    """
    print(f"getting dataset {dataset}")
    # artists, users, plays = get_lastfm()
    getdata = dataSets.get(dataset)
    if not getdata:
        raise ValueError(f"Unknown Model {dataset}")
    artists, users, plays = getdata()

    return artists, users, plays

### `thresholdLikes()`

In [8]:
def thresholdLikes(df, UIDmin, MIDmin):
    """
    Removes from data set those items having fewer than 


    """

    userCt = df.uid.unique().shape[0]
    itemCt = df.mid.unique().shape[0]
    sparsity = float(df.shape[0]) / float(userCt*itemCt) * 100
    print("Starting likes info")
    print("Number of users: {userCt}")
    print("Number of models: {itemCt}")
    print("Sparsity: {sparsity:4.3f}%")

    done = False
    while not done:
        starting_shape = df.shape[0]
        mid_counts = df.groupby('uid').mid.count()
        df = df[~df.uid.isin(mid_counts[mid_counts < MIDmin].index.tolist())]
        uid_counts = df.groupby('mid').uid.count()
        df = df[~df.mid.isin(uid_counts[uid_counts < UIDmin].index.tolist())]
        ending_shape = df.shape[0]
        if starting_shape == ending_shape:
            done = True
    
    assert(df.groupby('uid').mid.count().min() >= MIDmin)
    assert(df.groupby('mid').uid.count().min() >= UIDmin)
    
    userCt = df.uid.unique().shape[0]
    itemCt = df.mid.unique().shape[0]
    sparsity = float(df.shape[0]) / float(userCt*itemCt) * 100
    print('Ending likes info')
    print('Number of users: {}'.format(userCt))
    print('Number of models: {}'.format(itemCt))
    print('Sparsity: {:4.3f}%'.format(sparsity))

    return df

### `trainTestSplit()`



In [28]:
def trainTestSplit(ratings, splitCount, fraction=None):
    """
    Split recommendation data into train and test sets
    
    Params
    ------
    ratings : scipy.sparse matrix
        Interactions between users and items.
    splitCount : int
        Number of user-item-interactions per user to move
        from training to test set.
    fractions : float
        Fraction of users to split off some of their
        interactions into test set. If None, then all 
        users are considered.
    """

    # Note: likely not the fastest way to do things below.
    train = ratings.copy().tocoo()
    test = sparse.lil_matrix(train.shape)
    
    if fraction:
        try:
            userIndex = np.random.choice(
                np.where(np.bincount(train.row) >= splitCount * 2)[0], 
                replace=False,
                size=np.int32(np.floor(fraction * train.shape[0]))
            ).tolist()
        except:
            raise Exception(f"Not enough users with > {2*splitCount} "
                            f"interactions for fraction of {fraction}")
    else:
        userIndex = range(train.shape[0])
        
    train = train.tolil()

    for user in userIndex:
        testRatings = np.random.choice(ratings.getrow(user).indices, 
                                        size=splitCount, 
                                        replace=False)
        train[user, testRatings] = 0.
        # These are just 1.0 right now
        test[user, testRatings] = ratings[user, testRatings]
   
    
    # Test and training are truly disjoint
    assert(train.multiply(test).nnz == 0)
    return train.tocsr(), test.tocsr(), userIndex

### `calculateSimilarArtists()`

After fetching the data, applies the `similar_items` API of the models

In [10]:
def calculateSimilarArtists(output_filename, dataset, model_name="als"):
    """
    Generates a list of similar artists in lastfm by utilizing the
    'similar_items' api of the models
    """

    # Get the data
    artists, users, plays = getDatums(dataset)

    # create a model from the input data
    model = get_model(model_name)

    # if we're training an ALS based model, weight input for last.fm
    # by bm25
    if issubclass(model.__class__, AlternatingLeastSquares):
        # lets weight these models by bm25weight.
        logging.debug("weighting matrix by bm25_weight")
        plays = bm25_weight(plays, K1=100, B=0.8)

        # also disable building approximate recommend index
        model.approximate_recommend = False

    # this is actually disturbingly expensive:
    plays = plays.tocsr()

    logging.debug("training model %s", model_name)
    start = time.time()
    model.fit(plays)
    logging.debug("trained model '%s' in %0.2fs", model_name,
                  time.time() - start)

    # write out similar artists by popularity
    start = time.time()
    logging.debug("calculating top artists")

    user_count = np.ediff1d(plays.indptr)
    to_generate = sorted(np.arange(len(artists)), key=lambda x: -user_count[x])

    # write out as a TSV of artistid, otherartistid, score
    logging.debug("writing similar items")
    with tqdm.tqdm(total=len(to_generate)) as progress:
        with codecs.open(output_filename, "w", "utf8") as o:
            for artistid in to_generate:
                artist = artists[artistid]
                for other, score in model.similar_items(artistid, 11):
                    o.write("%s\t%s\t%s\n" % (artist, artists[other], score))
                progress.update(1)

    logging.debug("generated similar artists in %0.2fs",  time.time() - start)


### `calculateRecommendations()`

Generates artist recommendations for each user in the dataset

In [11]:
def calculateRecommendations(output_filename, model_name="als"):
    """ Generates artist recommendations for each user in the dataset """

    # Get the data
    artists, users, plays = getDatums(dataset)

    # create a model from the input data
    model = get_model(model_name)

    # train the model based off input params

    # if we're training an ALS based model, weight input for last.fm
    # by bm25
    if issubclass(model.__class__, AlternatingLeastSquares):
        # lets weight these models by bm25weight.
        logging.debug("weighting matrix by bm25_weight")
        plays = bm25_weight(plays, K1=100, B=0.8)

        # also disable building approximate recommend index
        model.approximate_similar_items = False

    # this is actually disturbingly expensive:
    plays = plays.tocsr()

    logging.debug("training model %s", model_name)
    start = time.time()
    model.fit(plays)
    logging.debug("trained model '%s' in %0.2fs", model_name, time.time() - start)

    # generate recommendations for each user and write out to a file
    start = time.time()
    user_plays = plays.T.tocsr()
    with tqdm.tqdm(total=len(users)) as progress:
        with codecs.open(output_filename, "w", "utf8") as o:
            for userid, username in enumerate(users):
                for artistid, score in model.recommend(userid, user_plays):
                    o.write("%s\t%s\t%s\n" % (username, artists[artistid],
                                              score))
                progress.update(1)
    logging.debug("generated recommendations in %0.2fs",  time.time() - start)

### `calculateMSE()`

In [12]:
def calculateMSE(model, ratings, userIndex=None):
    preds = model.predict_for_customers()
    if userIndex:
        return mean_squared_error(ratings[userIndex, :].toarray().ravel(),
                                  preds[user_index, :].ravel())

    return mean_squared_error(ratings.toarray().ravel(),
                              preds.ravel())

### `precisionAtK()`

In [13]:
def precisionAtK(model, ratings, k=5, userIndex=None):

    if not userIndex:
        userIndex = range(ratings.shape[0])
    ratings = ratings.tocsr()
    precisions = []

    # Note: line below may become infeasible for large datasets.
    predictions = model.predict_for_customers()

    for user in userIndex:
        # In case of large dataset, compute predictions row-by-row like below
        # predictions = np.array([model.predict(row, i) for i in xrange(ratings.shape[1])])
        topK = np.argsort(-predictions[user, :])[:k]
        labels = ratings.getrow(user).indices
        precision = float(len(set(topK) & set(labels))) / float(k)
        precisions.append(precision)

    return np.mean(precisions)

### `printLog()`

In [14]:
def printLog(row, header=False, spacing=12):
    top = ''
    middle = ''
    bottom = ''
    for r in row:
        top += '+{}'.format('-'*spacing)
        if isinstance(r, str):
            middle += '| {0:^{1}} '.format(r, spacing-2)
        elif isinstance(r, int):
            middle += '| {0:^{1}} '.format(r, spacing-2)
        elif isinstance(r, float):
            middle += '| {0:^{1}.5f} '.format(r, spacing-2)
        bottom += '+{}'.format('='*spacing)
    top += '+'
    middle += '|'
    bottom += '+'
    if header:
        print(top)
        print(middle)
        print(bottom)
    else:
        print(middle)
        print(top)

### `learningCurve()`

In [50]:
def learningCurve(model, train, test, epochs, k=5, userIndex=None):
    if not userIndex:
        userIndex = range(train.shape[0])
    previousEpoch = 0
    trainPrecision = []
    trainMSE = []
    testPrecision = []
    testMSE = []
    
    headers = ['epochs', 'p@k train', 'p@k test',
               'mse train', 'mse test']
    printLog(headers, header=True)
    
    for epoch in epochs:
        model.iterations = epoch - previousEpoch
        if not hasattr(model, 'user_vectors'):
            model.fit(train)
        else:
            model.fit_partial(train)
        trainMSE.append(calculateMSE(model, train, userIndex))
        trainPrecision.append(precisionAtK(model, train, k, userIndex))
        testMSE.append(calculateMSE(model, test, userIndex))
        testPrecision.append(precisionAtK(model, test, k, userIndex))
        row = [epoch, trainPrecision[-1], testPrecision[-1],
               trainMSE[-1], testMSE[-1]]
        printLog(row)
        previousEpoch = epoch
    return model, trainPrecision, trainMSE, testPrecision, testMSE

### `gridSearchLearningCurve()`

In [44]:
def gridSearchLearningCurve(baseModel, train, test, paramGrid,
                            userIndex=None, patk=5, epochs=range(2, 40, 2)):
    """
    "Inspired" (stolen) from sklearn gridsearch
    https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/model_selection/_search.py
    """
    curves = []
    keys, values = zip(*paramGrid.items())
    for v in itertools.product(*values):
        params = dict(zip(keys, v))
        thisModel = copy.deepcopy(baseModel)
        print_line = []
        for k, v in params.items():
            setattr(thisModel, k, v)
            print_line.append((k, v))

        print(' | '.join('{}: {}'.format(k, v) for (k, v) in print_line))
        _, patkTrain, MSEtrain, patkTest, MSEtest = learningCurve(thisModel, train, test,
                                                                  epochs, k=patk,
                                                                  userIndex=userIndex)
        curves.append({'params': params,
                       'patk': {'train': patkTrain, 'test': patkTest},
                       'MSE': {'train': MSEtrain, 'test': MSEtest}})
    return curves

## Run code



In [20]:
artists, users, plays = getDatums('lastfm')

type(artists), artists[:4]
type(users), users[:4]
type(plays), plays[:4]

getting dataset lastfm


(numpy.ndarray,
 array([' 2 ', ' 58725ab=>', ' 80lİ yillarin tÜrkÇe sÖzlÜ aŞk Şarkilari',
        ' amy winehouse'], dtype=object))

(numpy.ndarray,
 array(['00000c289a1829a808ac09c00daf10bc3c4e223b',
        '00001411dc427966b17297bf4d69e7e193135d89',
        '00004d2ac9316e22dc007ab2243d6fcb239e707d',
        '000063d3fe1cf2ba248b9e3c3f0334845a27a6bf'], dtype=object))

(scipy.sparse.csr.csr_matrix,
 <4x358868 sparse matrix of type '<class 'numpy.float32'>'
 	with 7 stored elements in Compressed Sparse Row format>)

In [19]:
artists.shape, users.shape, plays.shape

((292385,), (358868,), (292385, 358868))

In [24]:
100*plays.count_nonzero()/(artists.shape[0]*users.shape[0])

0.01671209636692248

In [29]:
train, test, userIndex = trainTestSplit(plays, 5, 0.2)

In [34]:
paramGrid = {'num_factors': [20, 40, 80],
              'regularization': [1e-3, 1e-1, 1e1],
              'alpha': [32, 64, 128]}

modelALS = getModel('als')

getting model als


In [49]:
curves = gridSearchLearningCurve(modelALS, train, test,
                                 paramGrid,
                                 userIndex=userIndex,
                                 patk=5)

num_factors: 20 | regularization: 0.001 | alpha: 32
+------------+------------+------------+------------+------------+
|   epochs   | p@k train  |  p@k test  | mse train  |  mse test  |
+============+============+============+============+============+


AttributeError: 'AlternatingLeastSquares' object has no attribute 'predict_for_customers'